# **PROJECT - rent**
# Part 1

### STUDENTS:


#### - Gal Leibovich 208238352
#### - Amit Mirzayev 206765869

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
def get_houses2(keywords):
    house_list = list()
    import requests
    from bs4 import BeautifulSoup
    url = "https://www.ad.co.il/nadlanrent?sp275=17413&keyword=" + keywords
    response = requests.get(url)
    if not response.status_code == 200:
        print ("not 200")
        return None
    try:
        results_page = BeautifulSoup(response.content, 'html.parser')
        cards = results_page.find('div',{'class':'cards-wrap s m l'})
        for card in cards.find_all('div',{'class':'card-block'}):
            house_link = "https://www.ad.co.il" + card.find('a').get('href')
            house_list.append(house_link)
        return house_list
    except Exception as e:
        print("An error occurred:", str(e))
        return None

In [3]:
get_houses2("חוף הצוק")

['https://www.ad.co.il/ad/16192577',
 'https://www.ad.co.il/ad/16191006',
 'https://www.ad.co.il/ad/16112050',
 'https://www.ad.co.il/ad/15290471',
 'https://www.ad.co.il/ad/16193327',
 'https://www.ad.co.il/ad/16191349',
 'https://www.ad.co.il/ad/16191454',
 'https://www.ad.co.il/ad/16191461',
 'https://www.ad.co.il/ad/16191137',
 'https://www.ad.co.il/ad/12999482',
 'https://www.ad.co.il/ad/12967883',
 'https://www.ad.co.il/ad/15611996',
 'https://www.ad.co.il/ad/15612021',
 'https://www.ad.co.il/ad/6641386']

In [4]:
import requests
import json
import urllib.parse

API_KEY = 'AIzaSyB0Q463457657657567'

def get_distance1(location1, location2):
    if "תל אביב" not in location1 and "Tel Aviv" not in location1:
        location1 += ", תל אביב"

    url = f"https://routes.googleapis.com/directions/v2:computeRoutes"
    
    headers = {
        'Content-Type': 'application/json',
        'X-Goog-Api-Key': API_KEY,
        'X-Goog-FieldMask': 'routes.distanceMeters,routes.duration'
    }

    body = {
        "origin": {
            "address": location1
        },
        "destination": {
            "address": location2
        },
        "travelMode": "DRIVE",
        "routingPreference": "TRAFFIC_AWARE"
    }

    response = requests.post(url, headers=headers, data=json.dumps(body)) 
    try:
        if response.status_code == 200:
            data = response.json()
            if 'routes' in data and len(data['routes']) > 0:
                distance_meters = data['routes'][0]['distanceMeters']
                return distance_meters
            else:
                print(f"No valid routes found for {location1} to {location2}")
                return None
        else:
            print("Error:", response.status_code, response.text)
            return None
    except Exception as e:
        print(f"Error processing the response: {str(e)}")
        return None


In [5]:
def get_house_info1(house_link):
    house_dict = dict()
    faci_list = []
    try:
        property_data = {
            "פרטי הנכס": None,
            "שכונה": None,
            "כתובת": None,
            "חדרים": None,
            "קומה": None,
            "שטח בנוי": None,
            "שטח גינה": None,
            "תאריך כניסה": None,
            "תשלומים בשנה": None,
            "ארנונה בחודש": None,
            "ועד בית בחודש": None
        }
        response = requests.get(house_link)
        if response.status_code != 200:
            return house_dict
        
        result_page = BeautifulSoup(response.content, 'html.parser')
        house_dict['address'] = result_page.find('div', class_="d-flex justify-content-between").find("h2").get_text(strip=True)

        proper = result_page.find('table', class_="table table-sm mb-4")
        for element in proper.find_all('tr'):
            cell = element.find_all("td")
            if cell[0].get_text(strip=True) in property_data:
                property_data[cell[0].get_text(strip=True)] = cell[1].get_text(strip=True)
       
    # עכשיו תחשב את המרחק ממרכז העיר
        city_center = "Dizengoff Square, Tel Aviv"
        address = house_dict['address']
        distance_from_center = get_distance1(address, city_center)
        house_dict['distance_from_center'] = distance_from_center 
        
        #החלפת שמות העמודות לאנגלית
        he_to_en_keys = {
            "פרטי הנכס": "property_type",
            "שכונה": "neighborhood",
            "כתובת": "address",
            "חדרים": "room_num",
            "קומה": "floor",
            "שטח בנוי": "area",
            "שטח גינה": "garden_area",
            "תאריך כניסה": "days_to_enter",
            "תשלומים בשנה": "num_of_payments",
            "ארנונה בחודש": "monthly_arnona",
            "ועד בית בחודש": "building_tax"
        }

        translated_property_data = {}
        for key, value in property_data.items():
            if key not in he_to_en_keys:
                continue
            new_key = he_to_en_keys[key]

            if new_key == 'room_num':
                try:
                    translated_property_data[new_key] = float(value)
                except:
                    translated_property_data[new_key] = np.nan

            elif new_key =='area':
                try:
                    translated_property_data[new_key] = int(value)
                except:
                    translated_property_data[new_key] = np.nan
                    
            elif new_key =='garden_area':
                try:
                    translated_property_data[new_key] = int(value)
                except:
                    translated_property_data[new_key] = 0

            elif new_key == 'days_to_enter':
                translated_property_data[new_key] = 0 if value == 'מיידית' else np.nan

            elif new_key == 'floor':
                try:
                    floor_parts = value.split()
                    translated_property_data['floor'] = int(floor_parts[0])
                    translated_property_data['total_floors'] = int(floor_parts[-1])
                except:
                    translated_property_data['floor'] = np.nan
                    translated_property_data['total_floors'] = np.nan

            elif new_key == 'monthly_arnona':
                try:
                    translated_property_data[new_key] = int(value.replace('₪', '').replace(',', '').strip())
                except:
                    translated_property_data[new_key] = 0

            elif new_key == 'building_tax':
                try:
                    translated_property_data[new_key] = int(value.replace('₪', '').replace(',', '').strip())
                except:
                    translated_property_data[new_key] = 0

            elif new_key == 'num_of_payments':
                try:
                    translated_property_data[new_key] = int(value)
                except:
                    translated_property_data[new_key] = 0

            else:
                translated_property_data[new_key] = value

        # תיאור
        desc_raw = result_page.find('p', class_="text-word-break").get_text(separator=' ', strip=True)
        desc_cleaned = desc_raw.replace('\n', ' ').replace('*', ' ')
        desc_cleaned = ' '.join(desc_cleaned.split()) 
        house_dict['description'] = desc_cleaned


        # מתקנים
        facilities = result_page.find('div', class_="card-icons flex-wrap d-flex h-100")
        for f in facilities.find_all('div'):
            class_list = f.get('class', [])
            faci_list.append(0 if 'disabled' in class_list else 1)

        house_dict['is_furnished'] = faci_list[1]
        house_dict['ac'] = faci_list[2]
        house_dict['has_parking'] = faci_list[3]
        house_dict['has_safe_room'] = faci_list[4]
        house_dict['has_balcon'] = faci_list[5]
        house_dict['handicap'] = faci_list[6]
        house_dict['has_bars'] = faci_list[7]
        house_dict['elevator'] = faci_list[8]
        house_dict['is_renovated'] = faci_list[9]
        house_dict['has_stotsge'] = faci_list[10]

        # תמונות
        try:
            house_dict['num_of_images'] = len(result_page.find('div', class_="col-12 d-flex mt-3 justify-content-center flex-wrap").find_all("div"))
        except:
            house_dict['num_of_images'] = 0

        # מחיר
        try:
            price_text = result_page.find('div', class_="d-flex justify-content-between").find_all("h2")[1].get_text(strip=True)
            house_dict['price'] = float(price_text.replace('₪', '').replace(',', '').strip())
        except:
            house_dict['price'] = np.nan

        # מיזוג נתונים
        merged_dict = {**translated_property_data, **house_dict}
        return merged_dict

    except Exception as e:
        print("An error occurred:", str(e))
        return None


In [6]:
def get_all_houses(keywords):
    results = list()
    all_houses = get_houses2(keywords)
    for house in all_houses:
        a = get_house_info1(house) 
        
        results.append(a)
    return(results)

In [7]:
neigb1 = get_all_houses("לינקולן")
neigb2 = get_all_houses("חוף הצוק")

all_data = neigb1 + neigb2

pd.DataFrame(all_data)

,property_type,neighborhood,address,room_num,floor,total_floors,area,garden_area,days_to_enter,num_of_payments,...,has_parking,has_safe_room,has_balcon,handicap,has_bars,elevator,is_renovated,has_stotsge,num_of_images,price
0,דירה,לינקולן,לינקולן 5,1.0,3,4,28,0,0.0,12,...,0,0,0,0,0,0,0,0,7,4600.0
1,סטודיו/לופט,לינקולן,כורש 9,1.0,4,4,25,0,0.0,12,...,0,0,0,0,0,0,0,0,7,3200.0
2,דירה,לינקולן,מאזה 75,4.0,3,3,120,0,0.0,12,...,0,0,1,0,0,0,0,1,9,2600.0
3,דירה,לינקולן,דרך מנחם בגין 33,4.0,11,20,98,0,0.0,12,...,1,1,1,0,0,1,0,1,6,11500.0
4,דירה,לינקולן,וילסון 9,3.0,2,4,90,0,NaN,12,...,0,0,0,0,0,0,0,0,0,10000.0
5,דירה,חוף הצוק,יוניצמן 21,1.0,3,7,32,0,NaN,12,...,1,0,1,1,0,1,0,1,4,4100.0
6,דירה,חוף הצוק,מיכאל נאמן 18,4.5,10,11,178,0,NaN,3,...,1,0,1,1,0,1,1,1,10,13900.0
7,דירת גן,חוף הצוק,יוניצ'מן 1,3.0,1,10,135,120,0.0,12,...,1,0,0,0,0,1,0,0,1,19500.0
8,דירה,חוף הצוק,יוניצ'מן 4,4.5,3,13,123,12,0.0,12,...,1,1,1,1,0,1,1,1,1,11500.0
9,דירה,חוף הצוק,יוניצמן,3.5,2,11,125,0,0.0,12,...,1,1,1,1,1,1,1,1,2,12500.0


In [8]:
df = pd.DataFrame(all_data)
desired_columns = [
    "property_type", "neighborhood", "address", "room_num", "floor", "area", "garden_area", "days_to_enter",
    "num_of_payments", "monthly_arnona", "building_tax", "total_floors", "description",
    "has_parking", "has_stotsge", "elevator", "ac", "handicap", "has_bars", "has_safe_room",
    "has_balcon", "is_furnished", "is_renovated", "price", "num_of_images", "distance_from_center"
]

df = df[desired_columns]

In [9]:
df.to_csv("tel_aviv.csv", index=False, encoding='utf-8-sig')